In [6]:
from selectolax.parser import HTMLParser
import re

# test1 = "/Users/sharjeelmustafa/Desktop/RA24_Testing/0000088053-06-000310.txt"
# test2 = "/Users/sharjeelmustafa/Desktop/RA24_Testing/0000088053-17-000170.txt"
# test3 = "/Users/sharjeelmustafa/Desktop/RA24_Testing/0000855887-16-000012.txt"
test4 = "/Users/sharjeelmustafa/Desktop/PARSE_TEST/TEMP/0001193125-21-287030.txt"

In [10]:
# REGEX
rm_graphic = r"<DOCUMENT>\n?<TYPE>GRAPHIC(.*\n)*?<\/DOCUMENT>"
rm_table = r"<table(.|\n)*?</table>"
rm_extranl = r"\n{3,}"
rm_empty = r"^\s*$"
rm_spaces = r" {2,}"

In [8]:
text = open(test4, 'r').read()

In [ ]:
print("Preprocess: len=" + str(len(text)))
# Remove GRAPHIC
text = re.sub(rm_graphic, "", text)
print("Postprocess: len=" + str(len(text)))

# Remove Tables
if len(text) > 1_000_000:
    text = re.sub(rm_table, "", text, flags=re.IGNORECASE)

# text = re.sub(rm_table, "", text, flags=re.IGNORECASE)
print("Postprocess: len=" + str(len(text)))

In [145]:
# print(text)

In [ ]:
html = HTMLParser(text)
html.unwrap_tags(['font', 'i', 'sub', 'b'])
html = html.root.traverse()

count = 0
output = ""

# temp
# html_parser = HTMLParser(html)
# html_parser.unwrap_tags(['div', 'i', "p"])
# print(html_parser.body.html)

print(text.lower().count("<td>"))

# for node in html:
    
#     if node.tag.lower() == "p": 
#         temp = node.text().replace("\n", " ")
#         # Use unicode, e.g., \xa0 is the Unicode character for &nbsp;
#         temp = temp.replace("\xa0", "\n")
#         temp = temp.replace("\u2003", "  ")
#         if len(temp) > 2: temp = temp.replace("\n", " ")

#         # Remove extra newlines
#         # print(node.tag, node.attributes)
#         # print(node.text())
#         output += temp.strip() + "\n"
#         count+=1
    
#     if node.tag.lower() == "div":
#         print("COUNT", count)
#         # print(node.text(deep=False))
#         temp = node.text(deep=False).replace("\n", " ")
#         print(temp)
#         if temp is None: continue
        
#         # Use unicode, e.g., \xa0 is the Unicode character for &nbsp;
#         temp = temp.replace("\xa0", "\n")
#         temp = temp.replace("\u2003", "  ")
#         if len(temp) > 2: temp = temp.replace("\n", " ")

#         # Remove extra newlines
#         # print(node.tag, node.attributes)
#         # print(node.text())
#         output += temp.strip() + "\n"
#         count+=1


    # Need to find a way to remove parents and only take the children
    # TODO
    # 1. Finish parse
    # 2, Rerun tests
    # 3. Package as API
    # 4. Deploy on droplet
    # 5. link to platform
    # 6. Testing of functionality
    # 7. Prepare email
    # Remove same paragraph breaks
    

In [ ]:
output = re.sub(rm_extranl, "\n\n", output)
output = re.sub(rm_empty, "", output, flags=re.MULTILINE)
output = re.sub(rm_spaces, " ", output)
open("output.txt", "w").write(output)

print(count)
# 4159 lines including nbsp
# 2014 lines excluding nbsp

In [2]:
import os 
def get_files(path, label, exts):
    files = []
    for root, _, filenames in os.walk(path):
        for filename in filenames:
            if label in filename: continue
            if filename.endswith(tuple(exts)):
                files.append(os.path.join(root, filename))
    return files

In [32]:
input_folder = "/Users/sharjeelmustafa/Desktop/PARSE_TEST/IN"
output_folder = "/Users/sharjeelmustafa/Desktop/PARSE_TEST/OUT"
# parse_entry(input_folder, output_folder, label='Extracted', ext=['.txt', '.html', '.htm'])

files = get_files(input_folder, "Extracted", ['.txt', '.html', '.htm'])

lengths = []
for file in files:
    lengths.append(len(open(file, 'r').read()))
    # lengths.append(len(open(file, 'r').read().split("\n")))

# what if I use number of rows instead of length of text

In [ ]:
import pandas as pd


df = pd.DataFrame(lengths, columns=['Length'])
# df.describe()

print(df.quantile([0.25, 0.5, 0.75, 0.9, 0.95, 0.99, 1.0]))

# Removing files over 10_000_000 takes: 08m 49s, 03 removed
# Removing files over 1_000_000 takes:  00m 40s, 80 removed
# Removing files over 7_000_000 takes:  05m 49s, 05 removed
# Removing files over 5_321_661 takes:  02m 46s, 14 removed


In [ ]:
# df.drop(df[df['Length'] > 150_000].index, inplace=True)
df.hist(column='Length', bins=100, figsize=(10, 6))

In [69]:
from selectolax.parser import HTMLParser
from bs4 import BeautifulSoup
import re

# REGEX
rm_graphic = r"<DOCUMENT>\n?<TYPE>GRAPHIC(.*\n)*?<\/DOCUMENT>"
rm_table = r"<table(.|\n)*?</table>"
rm_br = r"<br>"
rm_extranl = r"\n{3,}"
rm_empty = r"^\s*$"
rm_spaces = r" {2,}"
rm_sup = r"<sup[\w\n =\"-:;%>]*</sup>"
rm_extra = r"^[*|•|\s]*$"

rm_start = r"<(div|tr|table)[\w\n =\"-:;%>]*>"
rm_end = r"</(div|tr|table)>"

rm_data1 = r"<td[\w\n =\"-:;%>]*>"
rm_data2 = r"</td>"



file_name = '0000898432-11-000018'
file_path = f"/Users/sharjeelmustafa/Desktop/PARSE_TEST/TEMP/{file_name}.htm"

text = open(file_path, 'r').read()
text = re.sub(rm_graphic, "", text)
text = re.sub(rm_br, " ", text, flags=re.IGNORECASE)
text = re.sub(rm_sup, "", text, flags=re.IGNORECASE)

text = re.sub(rm_start, "<p>", text, flags=re.IGNORECASE)
text = re.sub(rm_end, "</p>", text, flags=re.IGNORECASE)

text = re.sub(rm_data1, "", text, flags=re.IGNORECASE)
text = re.sub(rm_data2, "", text, flags=re.IGNORECASE)



open(f"{file_name}-temp.txt", "w").write(text)
output = ""
# Selectolax

text = """
<p>
    <p>
    <hr align="left" noshade size="4" width="100%">
    </p>
    <font style="DISPLAY: inline; FONT-FAMILY: Times New Roman; FONT-SIZE: 12pt">Board Consideration of the Management and Sub-Advisory Agreements</font>
</p>

<p>&#160;</p>
"""


print(text)

html = HTMLParser(text)
# # Can't use unwrap, it is too slow

for i, p in enumerate(html.css('p')):
    print(p.text(deep=True))
    parent = p.parent
    if parent and parent.tag == 'p':
        # Unwrap the current <p> tag and insert its children into the parent
        for child in p.child_nodes:
            parent.insert_before(child, p)  # Insert each child before the current <p> tag
        p.decompose()  # Remove the <p> tag
    
    # tag = element.tag

    # if element.parent and element.parent.tag == 'p':
    #    print("active")
    #    tmp = element.child.unwrap('p')
    # else:
    #     tmp = element.text(deep=True)
    # print(f"Row {i} {tag}:\t{tmp}")
    continue
    # tmp = element.text(deep=True)
    

    tmp = re.sub("\t", " ", tmp)
    tmp = re.sub("\xa0", "\n", tmp)
    tmp = re.sub("\u2003", "  ", tmp)
    tmp = re.sub("\n", " ", tmp)


    tmp = re.sub(rm_extra, "", tmp)
    tmp = tmp.strip()
    print(f"Row {i} {element.tag}:\t{tmp}")
    if tmp: output += tmp + "\n"
    else: output += "\n"

output = re.sub(rm_extranl, "\n\n", output)
output = re.sub(rm_empty, "", output, flags=re.MULTILINE)
output = re.sub(rm_spaces, " ", output)
open(f"{file_name}.txt", "w").write(output)



<p>
    <p>
    <hr align="left" noshade size="4" width="100%">
    </p>
    <font style="DISPLAY: inline; FONT-FAMILY: Times New Roman; FONT-SIZE: 12pt">Board Consideration of the Management and Sub-Advisory Agreements</font>
</p>

<p>&#160;</p>


    

    


 


0